import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!pip install folium
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 92kB 10.7MB/s eta 0:00:01
Libraries imported.


Read the postcode.csv file that contains the latitude and longitud coordinates of each postcode 

In [2]:

import types

from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_c86b5050d77f4042be18c7537dd1e416 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='LBM71Vd1Lf5OGr38QZ8R7pneasDXqcCfZ7Mb8xQYG3F5',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_c86b5050d77f4042be18c7537dd1e416.get_object(Bucket='capstone-donotdelete-pr-sg9rftn3nrcx55',Key='postcode.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

neig_T = pd.read_csv(body,index_col=0)



In [3]:
neig_T.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
3,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
4,M3B,North York,Don Mills North,43.745906,-79.352188


Explore and cluster the neighborhoods in Toronto. 


Decide to work with only boroughs that contain the word Toronto.

Select boroughs that contain the word Toronto. named boroughs_w_T

In [4]:
neig_T.Borough

0            North York
1            North York
2          Queen's Park
3             Etobicoke
4            North York
5            North York
6             East York
7      Downtown Toronto
8                  York
9          East Toronto
10     Downtown Toronto
11                 York
12          Scarborough
13            East York
14     Downtown Toronto
15     Downtown Toronto
16          Scarborough
17           North York
18            East York
19          Scarborough
20            East York
21           North York
22           North York
23           North York
24           North York
25         East Toronto
26           North York
27           North York
28      Central Toronto
29      Central Toronto
30                 York
31           North York
32      Central Toronto
33            Etobicoke
34           North York
35      Central Toronto
36          Mississauga
37          Scarborough
38      Central Toronto
39          Scarborough
40     Downtown Toronto
41     Downtown 

In [5]:

z="Toronto"
boroughs_w_T=[]
for id in neig_T.Borough: 
    if z in id.split(" "):
        if id not in boroughs_w_T:
            boroughs_w_T.append(id)


Order the boroughs in alphabetic way

In [6]:
boroughs_w_T.sort()

Make a list of neighbourhoods corresponding to boroughs that contain the word Toronto

In [7]:
list_neig_w_T=[neig_T.loc[neig_T.Borough==y] for y in boroughs_w_T]

Build the dataframe of neighbourhoods with the word Toronto, named neig_w_T

In [8]:
len(list_neig_w_T)

4

In [9]:
neig_w_T=pd.DataFrame(columns=neig_T.columns)
for i in range(len(list_neig_w_T)): 
    neig_w_T=pd.concat([neig_w_T,pd.DataFrame(list_neig_w_T[i],columns=neig_T.columns)], ignore_index=True)

In [10]:
neig_w_T

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
4,M4S,Central Toronto,Davisville,43.704324,-79.388790
5,M5P,Central Toronto,"Forest Hill North,Forest Hill West",43.696948,-79.411307
6,M5R,Central Toronto,"The Annex,North Midtown,Yorkville",43.672710,-79.405678
7,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
8,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049
9,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


Examine the dataframe resulting

Count the the number of boroughs in the neig_w_T dataframe

In [11]:
number_boroughs=neig_w_T.groupby(['Borough']).size()
number_boroughs=pd.DataFrame(number_boroughs,columns=["Count"])
print(number_boroughs)

                  Count
Borough                
Central Toronto       9
Downtown Toronto     18
East Toronto          5
West Toronto          6


Count the number of neighbourhoods corresponding to each borough

In [12]:
number_neig=[]
for i in range(len(boroughs_w_T)):
    len_neig=0
    group_neig=neig_w_T.loc[neig_w_T.Borough==boroughs_w_T[i]]["Neighbourhood"].tolist()
    for y in group_neig:
        y=y.split(',')
        len_neig+=len(y)
    number_neig.append(len_neig)

number_neig=pd.DataFrame(number_neig, index=number_boroughs.index,columns=["Neighbourhoods"])
print(number_neig)

                  Neighbourhoods
Borough                         
Central Toronto               17
Downtown Toronto              37
East Toronto                   7
West Toronto                  13


Show a table with the information of number boroughs and its number of neighbourhoods

In [13]:
number_boroughs_neig=pd.concat([number_boroughs,number_neig],axis=1,ignore_index=False)

In [14]:
print(number_boroughs_neig)

                  Count  Neighbourhoods
Borough                                
Central Toronto       9              17
Downtown Toronto     18              37
East Toronto          5               7
West Toronto          6              13


In [15]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(boroughs_w_T),
        number_boroughs_neig.Neighbourhoods.sum()
    )
)


The dataframe has 4 boroughs and 74 neighborhoods.


Build the dataframe named neighbourhoods that contain the borough, latitude and longitude corresponding to each neighbourhood that appears in the neig_w_T dataframe 

In [16]:
neighbourhoods=pd.DataFrame(columns=neig_w_T.columns[1:])
for ii in range(neig_w_T.shape[0]):
    borough=neig_w_T.loc[ii,"Borough"]
    lat=neig_w_T.loc[ii,"Latitude"]
    lng=neig_w_T.loc[ii,"Longitude"]
    for group_neig in [neig_w_T.loc[ii,'Neighbourhood']]:
            for neighbourhood in group_neig.split(','):
                 neighbourhoods=pd.concat([neighbourhoods,pd.DataFrame([[borough,neighbourhood,lat,lng]],columns=neighbourhoods.columns)],
                                     ignore_index=True)

In [17]:
neighbourhoods

,Borough,Neighbourhood,Latitude,Longitude
0,Central Toronto,Lawrence Park,43.728020,-79.388790
1,Central Toronto,Roselawn,43.711695,-79.416936
2,Central Toronto,Davisville North,43.712751,-79.390197
3,Central Toronto,North Toronto West,43.715383,-79.405678
4,Central Toronto,Davisville,43.704324,-79.388790
5,Central Toronto,Forest Hill North,43.696948,-79.411307
6,Central Toronto,Forest Hill West,43.696948,-79.411307
7,Central Toronto,The Annex,43.672710,-79.405678
8,Central Toronto,North Midtown,43.672710,-79.405678
9,Central Toronto,Yorkville,43.672710,-79.405678


Use geopy library to get the latitude and longitude values of Toronto. Define an instance of the geocoder, with the user_agent named "to_explorer"

In [18]:
address = 'Toronto,Ontario, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.7170226, -79.4197830350134.


Create a map of Toronto with neighbourhoods superimposed on top

In [19]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough,neighbourhood in zip(neighbourhoods['Latitude'], neighbourhoods['Longitude'], 
    neighbourhoods['Borough'],neighbourhoods['Neighbourhood']):
            label = '{}, {}'.format(neighbourhood, borough)
            label = folium.Popup(label, parse_html=True)
            folium.CircleMarker(
                [lat, lng],
                radius=5,
                popup=label,
                color='blue',
                fill=True,
                fill_color='#3186cc',
                fill_opacity=0.7,
                parse_html=False).add_to(map_toronto)  
map_toronto   

In [20]:
map_toronto.save("map_Toronto.html")

Segment and cluster only the neighborhoods in Downtown Toronto, and then make a slice the original dataframe and create a new dataframe of the Downtown Toronto data.

In [21]:
Downtown_Toronto_data=neighbourhoods[neighbourhoods['Borough']=="Downtown Toronto"].reset_index(drop=True)
Downtown_Toronto_data.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Downtown Toronto,St. James Town,43.651494,-79.375418
1,Downtown Toronto,Berczy Park,43.644771,-79.373306
2,Downtown Toronto,Central Bay Street,43.657952,-79.387383
3,Downtown Toronto,Christie,43.669542,-79.422564
4,Downtown Toronto,Rosedale,43.679563,-79.377529


Let's get the geographical coordinates of Downtown Toronto.

In [22]:
address = 'Downtown Toronto, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.3808116451341.


As we did with all of Toronto city, let's visualizate Downtown Toronto  the neighborhoods in it.

In [23]:
# create map of Manhattan using latitude and longitude values
map_Downtown_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Downtown_Toronto_data['Latitude'], Downtown_Toronto_data['Longitude'], Downtown_Toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Downtown_Toronto)  
    
map_Downtown_Toronto



In [24]:
map_Downtown_Toronto.save("map_Downtown_Toronto.html")

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

Define Foursquare Credentials and Version

In [25]:
CLIENT_ID = 'X3INNHZFS0RE4YAMNP5CCG4UVUP4S5K1S4NZIIXKHN2UNXJW' # your Foursquare ID
CLIENT_SECRET = '20EOF0IP0MALLD0RO0MM1QOCLEHXC35L4U3JHMETTEF24YJR' # your Foursquare Secret

VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: X3INNHZFS0RE4YAMNP5CCG4UVUP4S5K1S4NZIIXKHN2UNXJW
CLIENT_SECRET:20EOF0IP0MALLD0RO0MM1QOCLEHXC35L4U3JHMETTEF24YJR


Let's explore the first neighborhood in our dataframe.
Get the neighborhood's name.

In [26]:
Downtown_Toronto_data.loc[0,'Neighbourhood']

'St. James Town'

Get the neighborhood's latitude and longitude values.

In [27]:
neighbourhood_latitude = Downtown_Toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = Downtown_Toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name =Downtown_Toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of St. James Town are 43.6514939, -79.3754179.


Now, let's get the top 100 venues that are in St. James Town within a radius of 500 meters.
First, let's create the GET request URL. Name your URL url.

In [28]:
# type your answer here
latitude=neighbourhood_latitude
longitude=neighbourhood_longitude
radius=500
LIMIT=100
url='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=X3INNHZFS0RE4YAMNP5CCG4UVUP4S5K1S4NZIIXKHN2UNXJW&client_secret=20EOF0IP0MALLD0RO0MM1QOCLEHXC35L4U3JHMETTEF24YJR&ll=43.6514939,-79.3754179&v=20180605&radius=500&limit=100'

Send the GET request and examine the resutls

In [29]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d88f744cc7d410037dcb30e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'St. Lawrence',
  'headerFullLocation': 'St. Lawrence, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 126,
  'suggestedBounds': {'ne': {'lat': 43.6559939045, 'lng': -79.36921018606671},
   'sw': {'lat': 43.646993895499996, 'lng': -79.3816256139333}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '583f54320923922e8141d360',
       'name': 'Crepe TO',
       'location': {'address': '52 Church Street',
        'lat': 43.650063279511016,
        'lng': -79.37458664523871,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65006327

Using this function from Foursquare

In [30]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean json and structure it into a pandas dataframe

In [31]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Crepe TO,Creperie,43.650063,-79.374587
1,Gyu-Kaku Japanese BBQ,Japanese Restaurant,43.651422,-79.375047
2,GEORGE Restaurant,Restaurant,43.653346,-79.374445
3,GoodLife Fitness Toronto 137 Yonge Street,Gym,43.651242,-79.378068
4,Terroni,Italian Restaurant,43.650927,-79.375602


In [32]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


Explore Neighbourhoods in Downtown Toronto.
Let's create a function to repeat the same process to all the neighborhoods in Downtown Toronto

In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called Downtown_Toronto_venues.

In [34]:
Downtown_Toronto_venues = getNearbyVenues(names=Downtown_Toronto_data['Neighbourhood'],
                                   latitudes=Downtown_Toronto_data['Latitude'],
                                   longitudes=Downtown_Toronto_data['Longitude']
                                  )
print(Downtown_Toronto_venues.shape)
Downtown_Toronto_venues.head()


St. James Town
Berczy Park
Central Bay Street
Christie
Rosedale
Stn A PO Boxes 25 The Esplanade
Church and Wellesley
Harbourfront
Regent Park
Ryerson
Garden District
Adelaide
King
Richmond
Harbourfront East
Toronto Islands
Union Station
Design Exchange
Toronto Dominion Centre
Commerce Court
Victoria Hotel
Harbord
University of Toronto
Chinatown
Grange Park
Kensington Market
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Cabbagetown
St. James Town
First Canadian Place
Underground city
(2510, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,St. James Town,43.651494,-79.375418,Crepe TO,43.650063,-79.374587,Creperie
1,St. James Town,43.651494,-79.375418,Gyu-Kaku Japanese BBQ,43.651422,-79.375047,Japanese Restaurant
2,St. James Town,43.651494,-79.375418,GEORGE Restaurant,43.653346,-79.374445,Restaurant
3,St. James Town,43.651494,-79.375418,GoodLife Fitness Toronto 137 Yonge Street,43.651242,-79.378068,Gym
4,St. James Town,43.651494,-79.375418,Terroni,43.650927,-79.375602,Italian Restaurant


Let's check how many venues were returned for each neighborhood

In [35]:
Downtown_Toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,14,14,14,14,14,14
Berczy Park,56,56,56,56,56,56
CN Tower,14,14,14,14,14,14
Cabbagetown,46,46,46,46,46,46
Central Bay Street,85,85,85,85,85,85
Chinatown,100,100,100,100,100,100
Christie,16,16,16,16,16,16
Church and Wellesley,90,90,90,90,90,90


Let's find out how many unique categories can be curated from all the returned venues

In [36]:
print('There are {} uniques categories.'.format(len(Downtown_Toronto_venues['Venue Category'].unique())))

There are 207 uniques categories.


Analize each neighbourhood

In [37]:
# one hot encoding
Downtown_Toronto_onehot = pd.get_dummies(Downtown_Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Downtown_Toronto_onehot['Neighbourhood'] = Downtown_Toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Downtown_Toronto_onehot.columns[-1]] + list(Downtown_Toronto_onehot.columns[:-1])
Downtown_Toronto_onehot = Downtown_Toronto_onehot[fixed_columns]

Downtown_Toronto_onehot.head()


,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,St. James Town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,St. James Town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,St. James Town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Examine the new dataframe size

In [38]:
Downtown_Toronto_onehot.shape

(2510, 208)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [39]:
Downtown_Toronto_grouped = Downtown_Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
Downtown_Toronto_grouped


,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.00,0.010000,0.010000,0.000000,0.02,0.0000,0.000000,0.0000,0.000000,0.020000,0.000000,0.040000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.01,0.020000,0.000000,0.000000,0.01,0.030000,0.010000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.080000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.030000,0.000000,0.00,0.010000,0.010000,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.020000,0.000000,0.000000,0.01,0.000000,0.01,0.000000,0.010000,0.000000,0.020000,0.010000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.030000,0.00,0.00,0.010000,0.010000,0.000000,0.010000,0.010000,0.010000,0.000000,0.010000,0.000000,0.00,0.010000,0.000000,0.

Let's confirm the new size

In [40]:
Downtown_Toronto_grouped.shape

(36, 208)

Let's print each neighborhood along with the top 5 most common venues

In [41]:
num_top_venues = 5

for hood in Downtown_Toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Downtown_Toronto_grouped[Downtown_Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
          venue  freq
0   Coffee Shop  0.08
1          Café  0.05
2    Steakhouse  0.04
3           Bar  0.04
4  Burger Joint  0.03


----Bathurst Quay----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport Terminal  0.14
3       Coffee Shop  0.07
4           Airport  0.07


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2  Italian Restaurant  0.04
3      Farmers Market  0.04
4                Café  0.04


----CN Tower----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport Terminal  0.14
3       Coffee Shop  0.07
4           Airport  0.07


----Cabbagetown----
                venue  freq
0         Coffee Shop  0.09
1          Restaurant  0.07
2  Italian Restaurant  0.04
3                Café  0.04
4                 Pub  0.04


----Central Bay Street----
                       venue  freq
0                Coffee Shop  0.14
1                

Let's put that into a pandas dataframe.
First, let's write a function to sort the venues in descending order.

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [43]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = Downtown_Toronto_grouped['Neighbourhood']

for ind in np.arange(Downtown_Toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Downtown_Toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Bar,Steakhouse,Restaurant,American Restaurant,Hotel,Burger Joint,Thai Restaurant,Cosmetics Shop
1,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Boutique,Boat or Ferry,Sculpture Garden,Coffee Shop,Bar,Harbor / Marina,Airport Food Court
2,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Café,Bakery,Italian Restaurant,Steakhouse,Beer Bar,Seafood Restaurant,Farmers Market
3,CN Tower,Airport Lounge,Airport Service,Airport Terminal,Boutique,Boat or Ferry,Sculpture Garden,Coffee Shop,Bar,Harbor / Marina,Airport Food Court
4,Cabbagetown,Coffee Shop,Restaurant,Café,Italian Restaurant,Park,Bakery,Pizza Place,Pub,General Entertainment,Indian Restaurant


Cluster Neighborhoods.
Run k-means to cluster the neighborhood into 5 clusters.

In [44]:
# set number of clusters
kclusters = 5

Downtown_Toronto_grouped_clustering = Downtown_Toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Downtown_Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:35] 

array([0, 1, 0, 1, 0, 0, 2, 4, 0, 0, 0, 0, 0, 2, 2, 0, 0, 1, 1, 2, 0, 1,
       1, 0, 0, 3, 0, 1, 0, 0, 0, 0, 0, 0, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood.

In [45]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Downtown_Toronto_merged = Downtown_Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each Neighbourhood
Downtown_Toronto_merged = Downtown_Toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Downtown_Toronto_merged.head() # check the last columns!

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Restaurant,Café,Italian Restaurant,Hotel,Bakery,Pizza Place,Breakfast Spot,Park,Gastropub
1,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Cheese Shop,Café,Bakery,Italian Restaurant,Steakhouse,Beer Bar,Seafood Restaurant,Farmers Market
2,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Café,Middle Eastern Restaurant,Italian Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Chinese Restaurant,Japanese Restaurant,Bubble Tea Shop
3,Downtown Toronto,Christie,43.669542,-79.422564,4,Grocery Store,Café,Park,Nightclub,Diner,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Convenience Store
4,Downtown Toronto,Rosedale,43.679563,-79.377529,3,Park,Playground,Trail,Building,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


Finally, let's visualize the resulting clusters

In [46]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Downtown_Toronto_merged['Latitude'], Downtown_Toronto_merged['Longitude'], Downtown_Toronto_merged['Neighbourhood'], Downtown_Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


In [47]:
map_clusters.save("map_Downtown_Toronto_clusters.html")

Examine Clusters.
Now, I can examine each cluster and determine the discriminating venue categories that distinguish each cluster.
Based on the defining categories, I will assign a name to each cluster.

Cluster 1 : Caffee Shop

In [48]:
    
    Downtown_Toronto_merged.loc[Downtown_Toronto_merged['Cluster Labels']==0,:]
 

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Restaurant,Café,Italian Restaurant,Hotel,Bakery,Pizza Place,Breakfast Spot,Park,Gastropub
1,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Cheese Shop,Café,Bakery,Italian Restaurant,Steakhouse,Beer Bar,Seafood Restaurant,Farmers Market
2,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Café,Middle Eastern Restaurant,Italian Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Chinese Restaurant,Japanese Restaurant,Bubble Tea Shop
5,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846,0,Coffee Shop,Restaurant,Café,Italian Restaurant,Beer Bar,Seafood Restaurant,Hotel,Cocktail Bar,Lounge,Gym
6,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Gay Bar,Japanese Restaurant,Sushi Restaurant,Restaurant,Men's Store,Hotel,Gastropub,Pub,Fast Food Restaurant
7,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Park,Pub,Café,Bakery,Gym / Fitness Center,Breakfast Spot,Restaurant,Theater,Mexican Restaurant
8,Downtown Toronto,Regent Park,43.654260,-79.360636,0,Coffee Shop,Park,Pub,Café,Bakery,Gym / Fitness Center,Breakfast Spot,Restaurant,Theater,Mexican Restaurant
9,Downtown Toronto,Ryerson,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Cosmetics Shop,Café,Italian Restaurant,Diner,Restaurant,Bakery,Plaza
10,Downtown Toronto,Garden District,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Cosmetics Shop,Café,Italian Restaurant,Diner,Restaurant,Bakery,Plaza
11,Downtown Toronto,Adelaide,43.650571,-79.384568,0,Coffee Shop,Café,Bar,Steakhouse,Restaurant,American Restaurant,Hotel,Burger Joint,Thai Restaurant,Cosmetics Shop


Cluster 2: Airport Lounge

In [49]:
Downtown_Toronto_merged.loc[Downtown_Toronto_merged['Cluster Labels']==1,:]

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,Downtown Toronto,CN Tower,43.628947,-79.39442,1,Airport Lounge,Airport Service,Airport Terminal,Boutique,Boat or Ferry,Sculpture Garden,Coffee Shop,Bar,Harbor / Marina,Airport Food Court
27,Downtown Toronto,Bathurst Quay,43.628947,-79.39442,1,Airport Lounge,Airport Service,Airport Terminal,Boutique,Boat or Ferry,Sculpture Garden,Coffee Shop,Bar,Harbor / Marina,Airport Food Court
28,Downtown Toronto,Island airport,43.628947,-79.39442,1,Airport Lounge,Airport Service,Airport Terminal,Boutique,Boat or Ferry,Sculpture Garden,Coffee Shop,Bar,Harbor / Marina,Airport Food Court
29,Downtown Toronto,Harbourfront West,43.628947,-79.39442,1,Airport Lounge,Airport Service,Airport Terminal,Boutique,Boat or Ferry,Sculpture Garden,Coffee Shop,Bar,Harbor / Marina,Airport Food Court
30,Downtown Toronto,King and Spadina,43.628947,-79.39442,1,Airport Lounge,Airport Service,Airport Terminal,Boutique,Boat or Ferry,Sculpture Garden,Coffee Shop,Bar,Harbor / Marina,Airport Food Court
31,Downtown Toronto,Railway Lands,43.628947,-79.39442,1,Airport Lounge,Airport Service,Airport Terminal,Boutique,Boat or Ferry,Sculpture Garden,Coffee Shop,Bar,Harbor / Marina,Airport Food Court
32,Downtown Toronto,South Niagara,43.628947,-79.39442,1,Airport Lounge,Airport Service,Airport Terminal,Boutique,Boat or Ferry,Sculpture Garden,Coffee Shop,Bar,Harbor / Marina,Airport Food Court


Cluster 3 : Cafe

In [50]:
Downtown_Toronto_merged.loc[Downtown_Toronto_merged['Cluster Labels']==2,:]

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Downtown Toronto,Harbord,43.662696,-79.400049,2,Café,Bookstore,Restaurant,Sandwich Place,Japanese Restaurant,Bar,Bakery,Nightclub,Sushi Restaurant,Pub
22,Downtown Toronto,University of Toronto,43.662696,-79.400049,2,Café,Bookstore,Restaurant,Sandwich Place,Japanese Restaurant,Bar,Bakery,Nightclub,Sushi Restaurant,Pub
23,Downtown Toronto,Chinatown,43.653206,-79.400049,2,Café,Vegetarian / Vegan Restaurant,Bar,Chinese Restaurant,Dumpling Restaurant,Mexican Restaurant,Bakery,Vietnamese Restaurant,Coffee Shop,Burger Joint
24,Downtown Toronto,Grange Park,43.653206,-79.400049,2,Café,Vegetarian / Vegan Restaurant,Bar,Chinese Restaurant,Dumpling Restaurant,Mexican Restaurant,Bakery,Vietnamese Restaurant,Coffee Shop,Burger Joint
25,Downtown Toronto,Kensington Market,43.653206,-79.400049,2,Café,Vegetarian / Vegan Restaurant,Bar,Chinese Restaurant,Dumpling Restaurant,Mexican Restaurant,Bakery,Vietnamese Restaurant,Coffee Shop,Burger Joint


Cluster 4 : Park

In [51]:
Downtown_Toronto_merged.loc[Downtown_Toronto_merged['Cluster Labels']==3,:]

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,Rosedale,43.679563,-79.377529,3,Park,Playground,Trail,Building,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


Cluster 5 : Grocery Store

In [52]:
Downtown_Toronto_merged.loc[Downtown_Toronto_merged['Cluster Labels']==4,:]

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Downtown Toronto,Christie,43.669542,-79.422564,4,Grocery Store,Café,Park,Nightclub,Diner,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Convenience Store
